# Model Building

better to plot with scatter plot, but many features so plot several scatter of one variable feature with all others kept constant

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
xgb.set_config(verbosity=0)
from sklearn.feature_selection import RFECV
import random
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score

In [3]:
# Drop first column of dataframe
def drop_first_col(df):
    return df.iloc[: , 1:]

train = pd.read_csv("dataset\\train.csv")
test = pd.read_csv("dataset\\test.csv")

train = drop_first_col(train)
test = drop_first_col(test)

## Test on VAR (Multi Variate Time Series)

In [11]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

avg_rmse = []
avg_r2 = []
for rid in train['RID'].unique():
    print('FOR RECORD',rid)
    
    VAR_train = (train.loc[train['RID']==rid]).drop(['RID','VISCODE'],axis=1)
    VAR_test = (test.loc[test['RID']==rid]).drop(['RID','VISCODE'],axis=1)

    model = VAR(endog=VAR_train)
    model_fit = model.fit(trend="nc")

    # make prediction on test set
    prediction = model_fit.forecast(model_fit.endog, steps=len(VAR_test))

    #converting predictions to dataframe
    pred = pd.DataFrame(index=range(0,len(prediction)),columns=[VAR_train.columns])
    for j in range(0,13):
        for i in range(0, len(prediction)):
            pred.iloc[i][j] = prediction[i][j]

    print("prediction :",'\n',pred['MMSE'],'\n')
    print("true value :",'\n',VAR_test['MMSE'],'\n')
    #check rmse
    rmse = np.sqrt(mean_squared_error(pred['MMSE'], VAR_test['MMSE']))
    r2 = r2_score(VAR_test['MMSE'], pred['MMSE'])
    print('rmse value for MMSE is : ', rmse)
    print('r2 square value for MMSE is : ', r2)
    avg_rmse.append(rmse)
    avg_r2.append(r2)

FOR RECORD 4.0
prediction : 
    MMSE
0  30.0
1  22.0 

true value : 
 0    27.0
1    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.9999999999993783
r2 square value for MMSE is :  -7.99999999999627
FOR RECORD 33.0
prediction : 
     MMSE
0   28.5
1  25.75 

true value : 
 2    25.0
3    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.531057091414539
r2 square value for MMSE is :  0.0
FOR RECORD 41.0
prediction : 
    MMSE
0  24.0
1  24.0 

true value : 
 4    24.0
5    23.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.7071067811865425
r2 square value for MMSE is :  -0.9999999999999716
FOR RECORD 42.0
prediction : 
         MMSE
0  17.321429
1  13.248724 

true value : 
 6    22.0
7    23.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  7.647757994606922
r2 square value for MMSE is :  -232.95280937629633
FOR RECORD 45.0
prediction : 
         MMSE
0  26.983784 

true value : 
 8    22.0
Name: MMSE, dtype: float64

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
 

 

rmse value for MMSE is :  4.983783843432263
r2 square value for MMSE is :  nan
FOR RECORD 51.0
prediction : 
    MMSE
0  29.0
1  29.0 

true value : 
 9     28.0
10    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.2360679774998773
r2 square value for MMSE is :  -4.000000000000391
FOR RECORD 57.0
prediction : 
         MMSE
0  22.763158
1  22.682133 

true value : 
 11    20.0
12    14.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  6.442611052811934
r2 square value for MMSE is :  -3.6119152419793874
FOR RECORD 77.0
prediction : 
    MMSE
0  16.0
1   8.5 

true value : 
 13    19.0
14    21.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  9.089829481348175
r2 square value for MMSE is :  -81.62499999998644
FOR RECORD 101.0
prediction : 
       MMSE
0    27.75
1  27.9375 

true value : 
 15    28.0
16    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.0846350100197313
r2 square value for MMSE is :  -0.9314236111110956
FOR RECORD 102.0
pred

C:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warnin

 19    18.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  2.032528744992524
r2 square value for MMSE is :  nan
FOR RECORD 112.0
prediction : 
         MMSE
0  27.833333
1      27.75 

true value : 
 20    29.0
21    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2090515107122877
r2 square value for MMSE is :  0.0
FOR RECORD 126.0
prediction : 
         MMSE
0  32.333333
1  37.333333 

true value : 
 22    26.0
23    22.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  11.730776236512526
r2 square value for MMSE is :  -33.40277777778174
FOR RECORD 150.0
prediction : 
       MMSE
0    28.75
1  28.5625 

true value : 
 24    28.0
25    27.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2255419719454983
r2 square value for MMSE is :  -5.007812500000242
FOR RECORD 176.0
prediction : 
    MMSE
0  27.0
1  27.0 

true value : 
 26    25.0
27    27.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.4142135623731
r2 square value for MMSE is :  -1

...

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, Unde

 

true value : 
 237    25.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.33333333333335
r2 square value for MMSE is :  nan
FOR RECORD 1309.0
prediction : 
    MMSE
0  30.0
1  31.0 

true value : 
 238    28.0
239    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  3.8078865529318073
r2 square value for MMSE is :  -13.499999999998881
FOR RECORD 1321.0
prediction : 
    MMSE
0  28.0
1  28.0 

true value : 
 240    29.0
241    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.0
r2 square value for MMSE is :  0.0
FOR RECORD 1352.0
prediction : 
     MMSE
0   28.5
1  29.25 

true value : 
 242    28.0
243    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  0.39528470752106876
r2 square value for MMSE is :  0.3749999999999325
FOR RECORD 1380.0
prediction : 
     MMSE
0   28.5
1  28.75 

true value : 
 244    29.0
245    27.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.2869537676233875
r2 square value for MMSE is :  -0.6562500000000

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_

prediction : 
         MMSE
0  18.357143
1  17.943878 

true value : 
 248    24.0
249    11.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  6.3268978250900565
r2 square value for MMSE is :  0.052552991973271346
FOR RECORD 1398.0
prediction : 
         MMSE
0  19.333333 

true value : 
 250    15.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  4.333333333333357
r2 square value for MMSE is :  nan
FOR RECORD 1419.0
prediction : 
     MMSE
0   33.5
1  14.75 

true value : 
 251    29.0
252    29.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  10.566752102702843
r2 square value for MMSE is :  0.0
FOR RECORD 1421.0
prediction : 
    MMSE
0  28.0
1  28.0 

true value : 
 253    27.0
254    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.5811388300841829
r2 square value for MMSE is :  -8.999999999999915
FOR RECORD 1423.0
prediction : 
         MMSE
0  27.991321 

true value : 
 255    26.0
Name: MMSE, dtype: float64 

rmse value for MMSE is :  1.991321

C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\marti\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:821: FutureWarning: trend 'nc' has been renamed to 'n' after 0.14 is released. Use 'n' now to avoid this warning.
  warnings.warn(
C:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [14]:
print('AVG RMSE:',sum(avg_rmse) / len(avg_rmse))
avg_r2 = [x for x in avg_r2 if str(x) != 'nan']
print('AVG R2:',sum(avg_r2) / len(avg_r2))

AVG RMSE: 3.584661485656106
AVG R2: -21.890297629419795


## Test on Autoregressive AR-X with XGBoost

https://cprosenjit.medium.com/multivariate-time-series-forecasting-using-xgboost-1728762a9eeb

In [9]:
import pandas as pd
from dateutil import relativedelta
from statsmodels.tsa.ar_model import AutoReg
# forecastWithAR: method to apply AutoRegression UV forecasting.
# dataset: input time series dataset in Pandas DataFrame format
# group_column: 'group by' attribute to be used for Pandas Func API
# forecast_max_date: end date of the forecasting horizon

def forecastWithAR(dataset, group_column, forecast_max_date):
    group_key = dataset[group_column].iloc[0]

    driver_max_date = dataset.Date.max()
    diff = relativedelta\
            .relativedelta(\
                           forecast_max_date, \
                           driver_max_date)
    forecastingPeriod = diff.months + diff.years * 12

    model_ag = AutoReg(endog = dataset["Value"], \
                     lags = 8, \
                     trend='c', \
                     seasonal = False, \
                     exog = None, \
                     hold_back = None, \
                     period = None, \
                     missing = 'none')
    fit_ag = model_ag.fit()
    predictStart = len(dataset)
    predictEnd = predictStart + \
                forecastingPeriod-1
    predictions = fit_ag.predict(\
                   start=predictStart, \
                   end=predictEnd, \
                   dynamic=False)

    # Assign column header
    predictions.name = "Value"

    # Convert Series to DataFrame
    pred_perf = predictions.to_frame()

    # Assign a column for predicted months
    prediction_start_month = \
    (dataset["Date"].iloc[-1] + \
     pd.DateOffset(months=1))\
    .strftime('%Y-%m-%d')

    pred_date_range = pd.date_range\
      (prediction_start_month, \
       periods=forecastingPeriod, \
       freq='MS')

    pred_perf[group_column] = group_key
    pred_perf['Date'] = pred_date_range

    # return prediction DataFrame
    return pred_perf[[group_column, "Date", "Value"]]

In [12]:
import numpy as np
from datetime import date
from dateutil import relativedelta
# Fetch the schema
schema = actualsDF.select("Driver", "Date", "Value").schema
# The column used for grouping the resultset
group_column = "Driver"
# Total prediction period i.e till 01-Oct-2022
forecast_max_date = date(2022,10,1)
# Python Function
def udf_uv_forecast (dataset:pd.DataFrame) -> pd.DataFrame:
    return forecastWithAR(dataset, group_column, forecast_max_date)
# Using Pandas Function API
forecastDF = actualsDF.groupby(group_column)\
                  .applyInPandas(udf_uv_forecast,\
                                 schema=schema)
display (forecastDF)

NameError: name 'actualsDF' is not defined